In [2]:
import os
import pandas as pd
from qer_utils.db import get_mongo_db
from qure_dicom_utils.dicom.series import geometry
from qer.ai.predictor.get_predictions import load_and_run_model ,load_model
from skimage.morphology import ball , disk , binary_erosion , binary_dilation, binary_closing, binary_opening
import h5py
db = get_mongo_db()
import numpy as np
import SimpleITK as sitk
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
from loguru import logger
import pickle

Unable to find logging config yaml. Configuring fallback values


In [3]:
import SimpleITK as sitk
from ipywidgets import widgets
from ipywidgets import interact
import matplotlib.pyplot as plt


def plot_scans(img_arr_list, title_list):
    cols = len(img_arr_list)
    num_slices = img_arr_list[0].shape[0]

    def callback(z=None):
        fig, ax = plt.subplots(1, cols, figsize=(5*cols, 5*cols), squeeze=False)
        for idx in range(cols):
            if idx == 0 : 
                ax[0][idx].imshow(img_arr_list[idx][z, :, :], cmap='gray')
            else :
                ax[0][idx].imshow(img_arr_list[idx][z, :, :])
            ax[0][idx].title.set_text(title_list[idx])
            ax[0][idx].axis('off')
            ax[0][idx].grid(False)

        fig.tight_layout()
        fig.show()

    interact(
        callback,
        z=widgets.IntSlider(value=0, min=0, max=(num_slices - 1), step=1)
    )

In [4]:
def get_dev_headct_arr(study_uid) : 
    base_path = '/cache/fast_data_nas8/qer/shubham/ich'
    path = os.path.join(base_path, study_uid)
    h5_data = path + '.h5'
    
    if os.path.exists(h5_data) :
        f2 = h5py.File(h5_data, 'r')

        if "mask" not in f2.keys():
            base_path = "/cache/fast_data_nas8/qer/shubham/ich_seg_train_val"
            path = os.path.join(base_path, study_uid)
            h5_data = path + '.h5'
            f2 = h5py.File(h5_data, 'r')
            image = f2['image']
            mask = f2["mask"]
            arr = np.array(image)
            mask = np.array(mask)
            f2.close()
            return arr ,mask
        
        image = f2['image']
        mask = f2["mask"]
        arr = np.array(image)
        mask = np.array(mask)
        f2.close()
        return arr, mask
    else:
        return None
    
def get_fda_spacing(study_uid): 
    rows = df_fda_test[df_fda_test.StudyUID == study_uid]
    if len(rows) > 0 :
        series_uid = rows.iloc[0].SeriesUID
        series_dict = db.dicoms.find_one({'_id' : series_uid })
        ftrs = geometry.get_features(series_dict)
        return (ftrs['delta_x'], ftrs['delta_y'], 5)
    
    
def get_dev_spacing(study_uid): 
    rows = df_hemorrhage_mask[df_hemorrhage_mask.StudyUID == study_uid]
    if len(rows) > 0 :
        series_uid = rows.iloc[0].SeriesUID_x
        series_dict = db.dicoms.find_one({'_id' : series_uid })
        ftrs = geometry.get_features(series_dict)
        return (ftrs['delta_x'], ftrs['delta_y'], 5)


def get_fda_headct_arr(study_uid):
    base_path = '/cache/fast_data_nas8/qer/shubham/ich_seg'
    path = os.path.join(base_path, study_uid)
    h5_data = path + '.h5'
    
    if os.path.exists(h5_data) :
        f2 = h5py.File(h5_data, 'r')
        image = f2['image']
        mask = f2["mask"]
        arr = np.array(image)
        mask = np.array(mask)
        f2.close()
        return arr, mask
    else:
        return None

In [5]:
from qer.utils.preprocessing.windowing import window_generator
brain_window = window_generator(80 , 40)
blood_window = window_generator(175 , 50)
bone_window = window_generator(3000 , 500)

In [16]:
df_fda = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/ICH_analyis_fda.csv")
df_val = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/ICH_analyis_val.csv")
df_train = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/ICH_analyis_train.csv")

In [25]:
random_train_uids = df_train[df_train["threshold"] == 0.5].sample(n= 7, random_state=12).StudyUID.values

In [33]:
random_val_uids = df_val[df_val["threshold"] == 0.5].sample(n= 8, random_state=90).StudyUID.values

In [37]:
random_fda_uids = df_fda[df_fda["threshold"] == 0.5].sample(n= 15, random_state=90).StudyUID.values

In [ ]:
random_fda_uids = df_fda[df_fda["threshold"] == 0.5].sample(n= 8, random_state=90).StudyUID.values

In [95]:
df_fda_test = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/qquantFDA_qure25k.csv")
fda_study_uids = list(df_fda_test.StudyUID.values)

In [6]:
uids0ml = []
for uid in tqdm(fda_study_uids) : 
    if os.path.exists(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5") : 
        h5f = h5py.File(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5" , "r")
        if np.array(h5f["acute_real_vol"]) == 0  :
            print(uid)
            uids0ml.append(uid)
        h5f.close()

  0%|          | 0/372 [00:00<?, ?it/s]

1.2.276.0.7230010.3.1.2.8323329.1346.1548393578.926171
1.2.276.0.7230010.3.1.2.8323329.1346.1548926903.1086689
1.2.840.113564.99.1.105263729736.54070.20191115216097.323543.2
1.2.840.113619.2.182.1080861679195.1574600414.4688478
1.2.840.113619.2.25.4.42240456.1573604637.315
1.2.840.113619.2.340.3.3523929760.378.1573470040.420
1.2.840.113619.2.404.3.397167661.400.1573911584.263
1.2.840.113696.513556.510.1357561.20191112205144
1.2.840.113711.514916.1.52896.595568019.26.2116281012.10
1.2.840.113704.1.111.2420.1577870635.1
1.2.840.114350.2.186.2.798268.2.302025995.1
1.2.410.200010.1221145.764.9387094.13429727.13429727
1.2.840.113619.2.290.3.1091212033.737.1558069955.845
1.2.840.113619.2.404.3.296508012.72.1551083430.193
1.2.840.113619.2.437.3.2215261088.102.1578560302.751
1.2.840.113619.2.437.3.2215261088.81.1593904570.318
1.2.840.113619.2.437.3.420137064.70.1596354183.183
1.2.840.113619.2.55.3.1762297613.729.1568604659.760
1.2.392.200036.9116.2.6.1.44063.1805253169.1603522061.843163


In [92]:
zeroml_uids = df_fda[(df_fda["threshold"] == 0.5) & (df_fda["StudyUID"].isin(uids0ml)) & (df_fda["dice_score"]>0)].StudyUID.values


In [74]:
len(df_fda[(df_fda["threshold"] == 0.5) & (df_fda["StudyUID"].isin(uids0ml)) & (df_fda["dice_score"] == 0) & (df_fda["pred_volume"] ==0 )])




8

In [78]:
len(df_val[(df_val["pred_volume"] ==0) & (df_val["threshold"] == 0.5)])

15

In [13]:
activation = torch.nn.Softmax(dim =1)

In [94]:
for uid in tqdm(zeroml_uids) : 
    if os.path.exists(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5") : 
        h5f = h5py.File(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5" , "r")
        print(uid)

        logits = np.array(h5f["logits"])[:,0,:,:]
        logits_out = torch.stack([torch.Tensor(-logits) , torch.Tensor(logits)],dim = 1)

        c = activation(logits_out).numpy()
        pred = c[:,1,:,:]
        mask = pred > 0.5
        arr ,gt_arr = get_fda_headct_arr(uid)
        plot_scans([brain_window(arr),gt_arr,logits,mask, logits > 0.3], ["scan","gt","logits","softmax on logits","model pred"])


  0%|          | 0/8 [00:00<?, ?it/s]

1.2.276.0.7230010.3.1.2.8323329.1346.1548393578.926171


interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

1.2.840.113619.2.25.4.42240456.1573604637.315


interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

...

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

1.2.840.113619.2.437.3.2215261088.81.1593904570.318


interactive(children=(IntSlider(value=0, description='z', max=39), Output()), _dom_classes=('widget-interact',…

1.2.392.200036.9116.2.6.1.44063.1805253169.1603522061.843163


interactive(children=(IntSlider(value=0, description='z', max=27), Output()), _dom_classes=('widget-interact',…

In [29]:
for uid in tqdm(random_train_uids) : 
    if os.path.exists(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5") : 
        h5f = h5py.File(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5" , "r")

        logits = np.array(h5f["logits"])
        logits_out = torch.stack([torch.Tensor(-logits) , torch.Tensor(logits)],dim = 1)

        c = activation(logits_out).numpy()
        pred = c[:,1,:,:]
        mask = pred > 0.5
        arr ,gt_arr = get_dev_headct_arr(uid)
        plot_scans([brain_window(arr),gt_arr,logits,mask, logits > 0.3], ["scan","gt","logits","softmax on logits","model pred"])


  0%|          | 0/7 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=39), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=43), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=35), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=27), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=33), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

In [34]:
for uid in tqdm(random_val_uids) : 
    if os.path.exists(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5") : 
        h5f = h5py.File(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5" , "r")

        logits = np.array(h5f["logits"])
        logits_out = torch.stack([torch.Tensor(-logits) , torch.Tensor(logits)],dim = 1)

        c = activation(logits_out).numpy()
        pred = c[:,1,:,:]
        mask = pred > 0.5
        arr ,gt_arr = get_dev_headct_arr(uid)
        plot_scans([brain_window(arr),gt_arr,logits,mask, logits > 0.3], ["scan","gt","logits","softmax on logits","model pred"])


  0%|          | 0/8 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=23), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=35), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=45), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=24), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=33), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=30), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

In [39]:
for uid in tqdm(random_fda_uids) : 
    if os.path.exists(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5") : 
        h5f = h5py.File(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5" , "r")

        logits = np.array(h5f["logits"])[:,0,:,:]
        logits_out = torch.stack([torch.Tensor(-logits) , torch.Tensor(logits)],dim = 1)

        c = activation(logits_out).numpy()
        pred = c[:,1,:,:]
        mask = pred > 0.5
        arr ,gt_arr = get_fda_headct_arr(uid)
        plot_scans([brain_window(arr),gt_arr,logits,mask, logits > 0.3], ["scan","gt","logits","softmax on logits","model pred"])


  0%|          | 0/15 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=33), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=34), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=32), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

...

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=29), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=27), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=25), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=27), Output()), _dom_classes=('widget-interact',…

In [81]:
train_study_uids = df_train[df_train["threshold"] == 0.5].StudyUID.values

In [82]:
uids0ml_train = []
for uid in tqdm(train_study_uids) : 
    if os.path.exists(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5") : 
        h5f = h5py.File(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5" , "r")
        if np.array(h5f["acute_real_vol"]) == 0  :
            uids0ml_train.append(uid)
        h5f.close()

  0%|          | 0/1516 [00:00<?, ?it/s]

In [85]:
len(uids0ml_train)

178

In [83]:
zeroml_uids_train = df_train[(df_train["threshold"] == 0.5) & (df_train["StudyUID"].isin(uids0ml_train)) & (df_train["dice_score"]==0)].StudyUID.values




In [84]:
len(zeroml_uids_train)

103

In [88]:
len(df_train[(df_train["threshold"] == 0.5) & (df_train["StudyUID"].isin(uids0ml_train)) & (df_train["dice_score"]>0) & (df_train["pred_volume"] > 0)])




75

In [91]:
df_train[(df_train["pred_volume"] == 0) & (df_train["threshold"] == 0.5)]

,StudyUID,Dataset,threshold,dice_score,pred_volume,abs_vol_error,dice_score_pp,pred_volume_pp,abs_vol_error_pp,dice_score_pp_no_dilation,pred_volume_pp_no_dilation,abs_vol_error_pp_no_dilation
427,1.2.840.113619.2.278.3.2831165748.949.13619138...,train,0.5,0.0,0.0,1.465051,0.0,0.0,1.465051,0.0,0.0,1.465051
655,1.2.840.113619.2.278.3.2831165736.312.13517400...,train,0.5,0.0,0.0,1.975296,0.0,0.0,1.975296,0.0,0.0,1.975296
1453,1.2.840.113619.2.81.290.1.1361.20140329.292857,train,0.5,0.0,0.0,0.033379,0.0,0.0,0.033379,0.0,0.0,0.033379
1776,1.2.840.113619.2.81.290.1.3224.20130930.222302,train,0.5,0.0,0.0,10.801552,0.0,0.0,10.801552,0.0,0.0,10.801552
2042,1.2.840.113619.2.334.3.2831165721.797.14035760...,train,0.5,0.0,0.0,1.415039,0.0,0.0,1.415039,0.0,0.0,1.415039
...,...,...,...,...,...,...,...,...,...,...,...,...
26609,1.2.840.113619.2.81.290.1.5993.20141222.221031,train,0.5,0.0,0.0,12.634990,0.0,0.0,12.634990,0.0,0.0,12.634990
27407,1.2.840.10008.1.474972.919176969.309843930.204...,train,0.5,0.0,0.0,0.954865,0.0,0.0,0.954865,0.0,0.0,0.954865
27882,1.3.6.1.4.1.25403.52240747607.2744.20150518060...,train,0.5,0.0,0.0,0.089237,0.0,0.0,0.089237,0.0,0.0,0.089237
28243,1.2.840.113619.2.278.3.2831165736.236.13543298...,train,0.5,0.0,0.0,3.908869,0.0,0.0,3.908869,0.0,0.0,3.908869


In [59]:
len(df_val[(df_val["threshold"] == 0.5) & (df_val["StudyUID"].isin(uids0ml_val))])

40

In [50]:
for uid in tqdm(zeroml_uids_val) : 
    if os.path.exists(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5") : 
        h5f = h5py.File(f"/cache/fast_data_nas8/qer/shubham/ICH_analysis/{uid}.h5" , "r")

        logits = np.array(h5f["logits"])
        logits_out = torch.stack([torch.Tensor(-logits) , torch.Tensor(logits)],dim = 1)

        c = activation(logits_out).numpy()
        pred = c[:,1,:,:]
        mask = pred > 0.5
        arr ,gt_arr = get_dev_headct_arr(uid)
        plot_scans([brain_window(arr),gt_arr,logits,mask, logits > 0.3], ["scan","gt","logits","softmax on logits","model pred"])


  0%|          | 0/15 [00:00<?, ?it/s]

interactive(children=(IntSlider(value=0, description='z', max=30), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=35), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=33), Output()), _dom_classes=('widget-interact',…

...

interactive(children=(IntSlider(value=0, description='z', max=35), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=31), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=34), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=33), Output()), _dom_classes=('widget-interact',…

interactive(children=(IntSlider(value=0, description='z', max=30), Output()), _dom_classes=('widget-interact',…

In [1]:
uid0 = "1.2.840.114350.2.186.2.798268.2.302025995.1"
uid1= "1.2.276.0.7230010.3.1.2.8323329.1346.1548393578.926171"

In [102]:
uids = [uid0 , uid1]

In [97]:
rows = df_fda_test[df_fda_test.StudyUID == uid0]
if len(rows) > 0 :
    series_uid = rows.iloc[0].SeriesUID
    series_dict = db.dicoms.find_one({'_id' : series_uid })

In [101]:
len([x["FilePath"] for x in series_dict["InstancesList"]])

64

In [6]:
df_ans = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/ICH_analyis_sdh_test_set.csv")
study_uids = df_ans[df_ans["pred_volume_pp"] < 0.2].StudyUID.values
df_test = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/ICH_dataset_test.csv")


In [7]:
df_ans

,StudyUID,old_volume,pred_volume,old_volume_pp,pred_volume_pp
0,1.2.840.113619.2.22.288.1.8523.20181130.202036,"<HDF5 dataset ""acute_real_vol"": shape (), type...",0.361204,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",2.371072
1,1.2.840.113619.2.22.288.1.8530.20181201.302306,"<HDF5 dataset ""acute_real_vol"": shape (), type...",0.009537,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",0.243187
2,1.2.840.113619.2.81.290.1.14251.20180730.250058,"<HDF5 dataset ""acute_real_vol"": shape (), type...",7.481262,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",19.640160
3,1.2.840.113619.2.81.290.1.14661.20181119.191432,"<HDF5 dataset ""acute_real_vol"": shape (), type...",3.176582,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",9.789154
4,1.2.840.113619.2.81.290.1.14694.20181126.325919,"<HDF5 dataset ""acute_real_vol"": shape (), type...",0.869036,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",3.623962
...,...,...,...,...,...
325,1.2.840.113619.2.22.288.1.5207.20121213.211634,"<HDF5 dataset ""acute_real_vol"": shape (), type...",9.366272,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",12.533662
326,1.2.840.113619.2.332.3.2831168304.466.14186133...,"<HDF5 dataset ""acute_real_vol"": shape (), type...",0.273255,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",0.390496
327,1.3.6.1.4.1.25403.52235783370.4220.20151210072...,"<HDF5 dataset ""acute_real_vol"": shape (), type...",7.611513,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",23.922920
328,1.2.840.113619.2.55.3.2831165733.497.138699982...,"<HDF5 dataset ""acute_real_vol"": shape (), type...",10.823000,"<HDF5 dataset ""acute_pp_vol"": shape (), type ""...",16.272051


In [15]:
df_test[df_test.StudyUID == study_uids[0]]

,StudyUID,PatientID,Status,Filepath,SeriesUID,Batch,Remarks,ICH,SDH,EDH,...,CALVARIAL,FRAC,CONFIRMED_HCT,POSTOP,NORMAL_BRAIN,UNCERTAIN_REPORT,ANY,crop,PATIENT_AGE,MANUFACTURER
930,1.3.12.2.1107.5.1.4.46190.4.0.636527019451476,6454,qure25k,test_set_fts/sampled/1.3.12.2.1107.5.1.4.46190...,1.3.12.2.1107.5.1.4.46190.4.0.636960227214541,test_set_fts,nii.gz attempted,0,1,0,...,0,0,1,0,0,0.0,1,NaN,60.0,SIEMENS


In [29]:
study_uids = ["1.2.840.113619.2.55.3.2831165748.830.1452395338.951"]

In [30]:
import shutil
for uid in tqdm(study_uids) : 
    rows = df_test[df_test.StudyUID == uid]
    if len(rows) > 0:
        series_uid = rows.iloc[0].SeriesUID
        series_dict = db.dicoms.find_one({'_id' : series_uid })
        filepaths = [x['FilePath'] for x in series_dict["InstancesList"]]
        filepaths = [filepath.replace( "/data_nas2/raw/" , "/data_nas3/raw/hct/")  for filepath in filepaths]

        new_filepaths = [filepath.replace( "/data_nas2/raw/" , "/data_nas3/raw/hct/")  for filepath in filepaths]

        invalid_paths = False
        for path in new_filepaths :
            if not os.path.exists(path) :
                invalid_paths = True
                break

        if invalid_paths : 
            new_filepaths = [filepath.replace( "/data_nas2/processed/HeadCT/" , "/data_nas3/processed/hct/")  for filepath in filepaths]



        # Source path
        for i in range(len(new_filepaths)): 
            source = new_filepaths[i]

            # Destination path
            dest_dir = f"/home/users/shubham.kumar/projects/ICH_classification_segmentation/tbr_test/{uid}"
            if not os.path.exists(dest_dir) :
                os.mkdir(dest_dir)
            destination = f"/home/users/shubham.kumar/projects/ICH_classification_segmentation/tbr_test/{uid}/{os.path.basename(source)}"

            # Copy the content of
            # source to destination

            try:
                shutil.copy(source, destination)
                print("File copied successfully.")

            # If source and destination are same
            except shutil.SameFileError:
                print("Source and destination represents the same file.")

            # If there is any permission issue
            except PermissionError:
                print("Permission denied.")

            # For other errors
            except:
                print("Error occurred while copying file.")


  0%|          | 0/1 [00:00<?, ?it/s]

File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.
File copied successfully.


In [6]:
pa = f"/home/users/shubham.kumar/projects/ICH_classification_segmentation/prod_test_cases/{uid0}"


In [7]:
uid1

'1.2.276.0.7230010.3.1.2.8323329.1346.1548393578.926171'

In [8]:
from skimage import morphology
def _post_process(mask, img_arr):
    mask = morphology.dilation(mask, morphology.ball(2))
    mask = np.array(
        [
            morphology.dilation(mask[i], morphology.disk(3))
            for i in range(mask.shape[0])
        ]
    )

    mask = mask * img_arr
    mask[mask < 40] = 0
    mask[mask > 100] = 0
    mask[mask > 0] = 1

    return mask

In [9]:
from qer.ai.predictor.get_predictions import load_and_run_model
import numpy as np
import SimpleITK as sitk
from os import path
import shutil
import zipfile
import glob
from qer.ai.utils.preprocessing import get_preprocessed_sitk_img
from qer.ai.predictor.utils.model_config import  default_model_configs

model_config  = default_model_configs["hemorrhages_quantification"]
print(model_config)


img = get_preprocessed_sitk_img(pa,model_config)

# img  = sitk.ReadImage("/home/users/shubham.kumar/projects/qureai/packages/python/qer/tests/ai/example_files/hemorrhages_quantification/zeroml_test.nii.gz") 
quant_output = load_and_run_model("hemorrhages_quantification",img,debug_mode = True)["results"]
acute_pp_vol = quant_output['quantification value']["acute_pp"]
acute_logits = quant_output['heatmaps']["acute_logits"]

threshold = 0.3
acute_old_mask = sitk.GetArrayFromImage(acute_logits)
acute_old_mask[acute_old_mask > threshold] = 1
acute_old_mask[acute_old_mask < threshold] = 0
print(acute_pp_vol)

acute_old_mask_pp = _post_process(acute_old_mask, sitk.GetArrayFromImage(img))
# acute_old_mask_pp*

a,b,c = img.GetSpacing()
print((np.sum(acute_old_mask_pp)*a*b*c)/1000)



<Instance of ModelConfig at address 140225865392080:
	name=hemorrhages_quantification
	class_name=HemorrhagesQuantificationModel
	init_kwargs={'checkpoint_path': 'hemorrhages_quantification/FDA_approved_mixed_mask.pth', 'mixed': True, 'img_size': 224, 'use_crop': True, 'threshold': 0.5}
	predict_kwargs={'sitk_img': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7f88dedd7270> >}
	dependencies=[]
	encrypted=False
	scan_preprocessing={'resample_thin': False, 'smooth_thin': True}
>
[2023-09-11 17:48:33 +0530] [INFO] Correcting image z spacing using median of slice positions from 2.65 to 2.65
[2023-09-11 17:48:33 +0530] [INFO] Loading model hemorrhages_quantification


ImageSeriesReader (0x2b009da0): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000873016



11.3257
3.991110964957827


In [8]:
df_test_sdh_vol =pd.read_csv("quant_SDH_volume_logits_correction_v4.csv")


In [18]:
df_quant_SDH_test = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/quant_SDH_q25k_outputs.csv")
df_test = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/ICH_dataset_test.csv")
a = df_test[df_test.StudyUID.isin(df_quant_SDH_test.StudyUID.values)].StudyUID.values
df_test_SDH = df_quant_SDH_test[df_quant_SDH_test["StudyUID"].isin(a)]


In [20]:
df_test_SDH[df_test_SDH["volume"] == 0]

,StudyUID,SeriesUID,volume,unit
656,1.2.840.113619.2.22.288.1.8523.20181130.202036,1.2.840.113619.2.22.288.1.8523.2.20181130.202149,0.0,ml
657,1.2.840.113619.2.22.288.1.8530.20181201.302306,1.2.840.113619.2.22.288.1.8530.2.20181201.302350,0.0,ml
828,1.2.840.113619.2.22.287.1.28929.20180715.194705,1.2.840.113619.2.22.287.1.28929.2.20180715.194747,0.0,ml
988,1.3.12.2.1107.5.1.4.34169.4.0.6061104312655291,1.3.12.2.1107.5.1.4.34169.4.0.6075330012385363,0.0,ml
1000,1.3.12.2.1107.5.1.4.46190.4.0.636527019451476,1.3.12.2.1107.5.1.4.46190.4.0.636960227214541,0.0,ml
...,...,...,...,...
6218,1.2.840.113619.2.55.3.252322801.331.1595058597...,1.2.840.113619.2.55.3.252322801.331.1595058597...,0.0,ml
6238,1.2.392.200036.9116.2.6.1.37.2429916378.160279...,1.2.392.200036.9116.2.6.1.37.2429916378.160279...,0.0,ml
6245,1.2.840.113619.2.404.3.1695215947.106.16025908...,1.2.840.113619.2.404.3.1695215947.106.16025908...,0.0,ml
6256,1.2.840.113711.169414.1.11948.622984925.26.211...,1.3.12.2.1107.5.1.4.55491.30000020101602562214...,0.0,ml


In [1]:
# dcm_path = "/home/users/shubham.kumar/projects/ICH_classification_segmentation/santhi/"

dcm_path = "/home/users/shubham.kumar/projects/qct_training_framework/notebooks/1.2.840.113619.2.22.287.1.28929.20180715.194705"



In [2]:
!nvidia-smi


Wed Sep 20 17:01:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.199.02   Driver Version: 470.199.02   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 33%   32C    P8    22W / 350W |      5MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:41:00.0 Off |                  N/A |
| 33%   

In [20]:
li = []
for uid in tqdm(study_uids) :
    try : 
        dcm_path = os.path.join("/home/users/shubham.kumar/projects/ICH_classification_segmentation/tbr_test" , uid)
        out = load_and_run_model("hemorrhages_v3" , dcm_path)
        quant_out = load_and_run_model("hemorrhages_quantification" , dcm_path)
        li.append({"StudyUID" : uid , "v3_score" : out['results']['scores']['ICH'] , "volume" : quant_out['results']['quantification value']['ICH']})
    except Exception as e :
        print(e)
        continue
    except KeyboardInterrupt :
        break

    

  0%|          | 0/22 [00:00<?, ?it/s]

[2023-09-20 18:19:14 +0530] [INFO] Loading model hemorrhages_v3
list index out of range
[2023-09-20 18:19:14 +0530] [INFO] Loading model hemorrhages_v3
[2023-09-20 18:19:15 +0530] [INFO] Correcting image z spacing using median of slice positions from 5.04 to 5.04


ImageSeriesReader (0xe740ac40): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.000514286



[2023-09-20 18:19:17 +0530] [INFO] Loading model hemorrhages_quantification


Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.


...

ImageSeriesReader (0xe740ac40): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.00180645



[2023-09-20 18:20:30 +0530] [INFO] Loading model hemorrhages_v3
[2023-09-20 18:20:31 +0530] [INFO] Correcting image z spacing using median of slice positions from 5.00 to 5.00
[2023-09-20 18:20:31 +0530] [INFO] Loading model hemorrhages_quantification


ImageSeriesReader (0xe740ac40): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.00518519



[2023-09-20 18:20:32 +0530] [INFO] Correcting image z spacing using median of slice positions from 5.00 to 5.00


ImageSeriesReader (0xe740ac40): Non uniform sampling or missing slices detected,  maximum nonuniformity:0.00518519



In [26]:
pd.DataFrame(li).iloc[0].StudyUID

'1.2.840.113619.2.55.3.2831165748.830.1452395338.951'

In [9]:
out['results']['scores']['ICH']


{'ICH': 0.9605829119682312}

In [10]:
quant_out = load_and_run_model("hemorrhages_quantification" , dcm_path)

[2023-09-20 17:02:10 +0530] [INFO] Loading model hemorrhages_quantification
[2023-09-20 17:02:12 +0530] [INFO] Correcting image z spacing using median of slice positions from 6.39 to 5.00


ImageSeriesReader (0xe3a89070): Non uniform sampling or missing slices detected,  maximum nonuniformity:6.38889



In [11]:
quant_out['results']['quantification value']['ICH']

{'results': {'heatmaps': {'ICH': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'std::vector< itk::simple::Image >::value_type *' at 0x7f1a3f9cc390> >},
  'quantification value': {'ICH': 1.031160143432628},
  'quantification units': {'ICH': 'ml'}}}

In [16]:
from qer.utils.imageoperations.resampler import load_raw_sitk_img

In [17]:
arr = sitk.GetArrayFromImage(load_raw_sitk_img(dcm_path))

[2023-09-18 21:18:10 +0530] [INFO] Correcting image z spacing using median of slice positions from 0.75 to 0.75


In [25]:
oo = quant_out["results"]["heatmaps"]["ICH"]

In [26]:
plot_scans([brain_window(arr) , sitk.GetArrayFromImage(oo)] , ["" , ""]  )

interactive(children=(IntSlider(value=0, description='z', max=210), Output()), _dom_classes=('widget-interact'…

In [8]:
blend_out = load_and_run_model("hemorrhages_blend" , dcm_path)

[2023-09-18 21:14:05 +0530] [INFO] Loading model hemorrhages_blend
[2023-09-18 21:14:06 +0530] [INFO] Correcting image z spacing using median of slice positions from 0.75 to 0.75
[2023-09-18 21:14:06 +0530] [INFO] Loading model hemorrhages_v3
[2023-09-18 21:14:08 +0530] [INFO] Loading model hemorrhages
[2023-09-18 21:14:08 +0530] [INFO] Resampling image; original thickness: 0.75; target thickness:  5.25
[2023-09-18 21:14:09 +0530] [INFO] Loading model hemorrhages_vjun23
[2023-09-18 21:14:09 +0530] [INFO] Loading pretrained weights from url (https://download.pytorch.org/models/resnet18-5c106cde.pth)


Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [10]:
blend_out

{'results': {'scores': {'ICH': 0.5047220413883527}}}

In [22]:
subtypes_out = load_and_run_model("hemorrhages_subtypes" , dcm_path)

[2023-09-20 14:45:33 +0530] [INFO] Loading model hemorrhages_subtypes
[2023-09-20 14:45:37 +0530] [INFO] Correcting image z spacing using median of slice positions from 0.75 to 0.75
[2023-09-20 14:45:37 +0530] [INFO] Resampling image; original thickness: 0.75; target thickness:  5.25


In [23]:
subtypes_out

{'results': {'scores': {'IPH': 0.2849116623401642,
   'IVH': 0.13028332591056824,
   'SAH': 0.48488178849220276,
   'SDH': 0.543889582157135,
   'EDH': 0.07927291095256805},
  'slice_scores': {'IPH': tensor([-26.5221, -23.5581, -21.9383, -17.8051, -10.4454,  -6.4038,  -5.7627,
            -3.9698,  -1.9539,  -2.1181,  -0.5817,  -0.4912,  -0.5231,  -0.1522,
            -0.4841,   0.2147,  -0.7072,  -0.2804,  -0.6144,  -0.5005,  -2.8482,
            -3.7969,  -3.4757,  -4.4790,  -6.6325,  -7.7006, -11.0229, -12.7239,
           -14.5327,     -inf]),
   'IVH': tensor([-14.8211, -12.9666, -11.7993,  -9.3912,  -5.6782,  -3.3055,  -2.8759,
            -2.2885,  -1.2560,  -1.4160,  -0.6732,  -0.8387,  -1.0863,  -0.7978,
            -0.1519,  -1.0344,  -1.2660,  -0.7494,  -1.0114,  -0.5753,  -2.3547,
            -3.0854,  -2.8602,  -3.5442,  -4.3623,  -4.7406,  -6.0132,  -7.3775,
            -8.8265,     -inf]),
   'SAH': tensor([-12.5325, -11.8497, -10.7486,  -9.2783,  -6.0791,  -4.0338,  -3.

In [ ]:
SDH TEST case - 5000 0-ml cases pick that

In [32]:
k  = {}

In [35]:
k["a"] = 1

In [33]:
if isinstance(k, dict) :
    print("yes")

yes


In [36]:
len(k)

1

In [41]:
list(k.items())[0]

('a', 1)

In [46]:
list(k.keys())[0]

'a'

In [47]:
original_dict = {
    "teacher1": "John",
    "teacher2": "Alice",
    "student_math": "Bob",
    "student_english": "Eve",
}

# Remove items with keys starting with "teacher"
keys_to_remove = [key for key in original_dict.keys() if key.startswith("teacher")]
for key in keys_to_remove:
    original_dict.pop(key)

# Rename keys starting with "student_" by replacing the label key
for key in list(original_dict.keys()):
    if key.startswith("student_"):
        label_key = key.split("_")[1]
        original_dict[f"student_{label_key}"] = original_dict.pop(key)

print(original_dict)


{'student_math': 'Bob', 'student_english': 'Eve'}


In [50]:
my_dict = {
    "a": "value"
}

# Get the value from the dictionary
value = next(iter(my_dict.keys()))

print(value)


a


In [4]:
from typing import Optional, List, Dict
import numpy as np
import pandas as pd
import os
import os
import ast
from tqdm.auto import tqdm
import h5py
from loguru import logger
import random
import SimpleITK as sitk

root_dir = "/data_nas3/processed/hct/Cache/all_imgs_cache.dcm/"

df_hemorrhage_mask = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/final_train_val_bleed_pos_seg_dataset.csv")
df_hemorrhage_mask = df_hemorrhage_mask[df_hemorrhage_mask.POSTOP==0]
df_hemorrhage_mask = df_hemorrhage_mask[df_hemorrhage_mask.Intensity=="acute"]
study_uids_having_bleed_mask = list(set(df_hemorrhage_mask.StudyUID.values))

df_bleed_seg_bbox = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/bleed_seg_bbox.csv")
df_normal = pd.read_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/train_normal_ncct.csv")
normal_study_uids = list(df_normal.sample(5000,random_state=85).StudyUID.values)


df_train = pd.read_csv("/home/users/shubham.kumar/projects/ICH_classification_segmentation/ICH_dataset_train.csv")
df_train = df_train[df_train["StudyUID"].isin(study_uids_having_bleed_mask)]

def get_mls_icv(sitk_img):
    output = load_and_run_model("mls_helper_icv_quant",sitk_img)
    icv_mask = output['results']['heatmaps']['ICV']
    icv_mask_arr = sitk.GetArrayFromImage(icv_mask)
#     midline_output = load_and_run_model("mls_quantification" ,sitk_img)
#     mls_arr = sitk.GetArrayFromImage(midline_output['results']['heatmaps']['MLS'])
#     return mls_arr , icv_mask_arr
    return icv_mask_arr

def _load_cls_data(df: pd.DataFrame, phase: str):
    data_dict = []

    for ind, row in tqdm(df.iterrows()):
        temp_dict = {}
        if phase == "test":
            base_path = "/cache/fast_data_nas8/qer/shubham/ich_seg"
            path = os.path.join(base_path, row["StudyUID"])
            temp_dict["datapath"] = path + ".h5"
            temp_dict["study_uid"] = row["StudyUID"]
        else:
            if row["ANY"] == 100:
                continue
            temp_dict["scan_label"] = row["ANY"]
            temp_dict["study_uid"] = row["StudyUID"]
            temp_dict["IPH"] = row["ICH"] or row["CONT"]
            temp_dict["SDH"] = row["SDH"]
            temp_dict["SAH"] = row["SAH"]
            temp_dict["EDH"] = row["EDH"]
            temp_dict["IVH"] = row["IVH"]
            base_path = "/cache/fast_data_nas8/qer/shubham/ich"
            path = os.path.join(base_path, row["StudyUID"])
            temp_dict["datapath"] = path + ".h5"
            if temp_dict["scan_label"] == 1 and temp_dict["study_uid"] in study_uids_having_bleed_mask: 
                try : 
                    f2 = h5py.File(temp_dict["datapath"], "r+")
                    if "mask" not in f2.keys():
                        base_path = "/cache/fast_data_nas8/qer/shubham/ich_seg_train_val"
                        path = os.path.join(base_path, row["StudyUID"])
                        temp_dict["datapath"] = path + ".h5"

                        # row = df_bleed_seg_bbox.loc[df_bleed_seg_bbox["StudyUID"]== temp_dict["study_uid"]]
                        # if len(row) > 0:
                        #     temp_dict["lower"] = int(row.z1.values[0])
                        #     temp_dict["upper"] = int(row.z2.values[0])

                    f2.close()
                except:
                    logger.debug(temp_dict["datapath"])
                    continue

            if row["StudyUID"] in normal_study_uids :
                base_path = "/cache/fast_data_nas8/qer/shubham/new_test"
                valid_paths = []
                study_uid =  row["StudyUID"]
                for perp_dist in range(0,30) : 
                    path = os.path.join(base_path, f"{study_uid}_{perp_dist}.h5" )
                    if os.path.exists(path) :
                        valid_paths.append(path)
                if len(valid_paths) > 0 : 
                    temp_dict["datapath"] = random.choice(valid_paths)
                    temp_dict["SDH"] = row["SDH"]
                    temp_dict["scan_label"] = 1
                else :
                    continue
            try:
                temp_dict["crop"] = np.array(ast.literal_eval(row["crop"]))
            except:
                temp_dict["crop"] = None

        if os.path.exists(temp_dict["datapath"]):
            if temp_dict["datapath"] in ["/cache/fast_data_nas8/qer/shubham/ich/1.2.840.113619.2.55.3.2831165742.939.1462502069.683.h5",
                                         "/cache/fast_data_nas8/qer/shubham/ich/1.2.840.113619.2.81.290.1.930.20140413.223950.h5",
                                         "/cache/fast_data_nas8/qer/shubham/test/1.3.6.1.4.1.25403.52234692458.1372.20150116103719.1_7.h5"]:
                continue
            data_dict.append(temp_dict)
    return data_dict



data_dict = _load_cls_data(df_train, "train")

for data in tqdm(data_dict) :
    try :
        
        datapath = data["datapath"]
        f2 = h5py.File(datapath, "r+")
        if "icv_mask" in f2.keys() :
            continue
        arr = np.array(f2["image"])
        img = sitk.GetImageFromArray(arr)
        img.SetSpacing((1,1,5))
        icv_arr = get_mls_icv(img)


#         slices_to_pick = np.unique(np.argwhere(mls_arr ==1)[:,0])
#         random.shuffle(slices_to_pick)
#         slice_to_pick = slices_to_pick[0]
#         req_mls_arr = mls_arr[slice_to_pick]

#         ml_coordinates = np.argwhere(req_mls_arr==1)
#         (x1,y1) = ml_coordinates[0]
#         (x2,y2) = ml_coordinates[-1]

#         # Calculate the slope (m) and y-intercept (b)
#         m = (y2 - y1) / (x2 - x1)
#         b = y1 - m * x1

#         # Create an empty mask
#         seperator_mask = np.zeros(req_mls_arr.shape, dtype=np.uint8)

#         for x in range(req_mls_arr.shape[0]):
#             for y in range(req_mls_arr.shape[1]):
#                 if y >= m * x + b:
#                     seperator_mask[x, y] = 2
#                 else :
#                     seperator_mask[x, y] = 1

        dseta = f2.create_dataset("icv_mask", data= icv_arr)
#         dseta = f2.create_dataset("hemi_seperator_mask", data= seperator_mask)

        f2.close()
    except KeyboardInterrupt:
        break
    except Exception as e :
        logger.debug(e)

    

0it [00:00, ?it/s]

In [ ]:
for data in tqdm(data_dict) :
    try :
        
        datapath = data["datapath"]
        f2 = h5py.File(datapath, "r+")
        if "icv_mask" in f2.keys() :
            continue
        arr = np.array(f2["image"])
        img = sitk.GetImageFromArray(arr)
        img.SetSpacing((1,1,5))
        icv_arr = get_mls_icv(img)


#         slices_to_pick = np.unique(np.argwhere(mls_arr ==1)[:,0])
#         random.shuffle(slices_to_pick)
#         slice_to_pick = slices_to_pick[0]
#         req_mls_arr = mls_arr[slice_to_pick]

#         ml_coordinates = np.argwhere(req_mls_arr==1)
#         (x1,y1) = ml_coordinates[0]
#         (x2,y2) = ml_coordinates[-1]

#         # Calculate the slope (m) and y-intercept (b)
#         m = (y2 - y1) / (x2 - x1)
#         b = y1 - m * x1

#         # Create an empty mask
#         seperator_mask = np.zeros(req_mls_arr.shape, dtype=np.uint8)

#         for x in range(req_mls_arr.shape[0]):
#             for y in range(req_mls_arr.shape[1]):
#                 if y >= m * x + b:
#                     seperator_mask[x, y] = 2
#                 else :
#                     seperator_mask[x, y] = 1

        dseta = f2.create_dataset("icv_mask", data= icv_arr)
#         dseta = f2.create_dataset("hemi_seperator_mask", data= seperator_mask)

        f2.close()
    except KeyboardInterrupt:
        break
    except Exception as e :
        logger.debug(e)

    

  0%|          | 0/1515 [00:00<?, ?it/s]

[2023-09-24 11:46:40 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:46:49 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:46:52 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:47:27 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:47:32 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:47:41 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:47:46 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:47:49 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:47:52 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:48:20 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:48:23 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:50:16 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:50:19 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:50:23 +0530] [INFO] Loading model mls_helper_icv_quant
[2023-09-24 11:50:25